In [24]:
import random
import numpy as np

# Charger la matrice d'adjacence et les points de collecte à partir du fichier CSV
def load_adjacency_matrix_and_collecte(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Supprimer les lignes vides
    lines = [line.strip() for line in lines if line.strip()]
    
    # La dernière ligne contient les points de collecte (exemple : 3;5)
    points_line = lines[-1]
    points_collecte = list(map(int, points_line.split(';')))
    
    # La matrice d'adjacence est dans toutes les autres lignes
    matrix = []
    for line in lines[:-1]:
        try:
            row = list(map(int, line.split(';')))
            matrix.append(row)
        except ValueError:
            print(f"Erreur de format dans la ligne : {line}. Elle sera ignorée.")
    
    return np.array(matrix), points_collecte

# Variables globales
start_node = 1  # On commence avec la ville 1

# Fonction de validation des chemins
def is_valid_path(path, matrix, collecte):
    if len(set(path)) != len(matrix):  # Il doit y avoir une ville pour chaque nœud
        return False
    if path[0] != start_node:  # doit commencer par la ville 1
        return False
    if path[-1] != start_node:  # doit finir par la ville 1
        return False
    if path[1] not in collecte:  # Le deuxième point doit être un point de collecte
        return False
    if path[-2] in collecte:  # L'avant-dernière ville ne doit pas être un point de collecte
        return False
    for i in range(len(path) - 1):
        if matrix[path[i] - 1, path[i+1] - 1] == -1:  # chemin bloqué (ajusté pour indexation à partir de 1)
            return False
    return True

# Génération d'une population initiale
def generate_initial_population(matrix, collecte, size=20):
    global start_node
    n = len(matrix)
    population = []
    
    while len(population) < size:
        # Créer une liste des villes disponibles (excluant la ville 0)
        remaining_nodes = list(range(1, n))  # Commence à 1 pour éviter la ville 0
        random.shuffle(remaining_nodes)
        path = [start_node] + remaining_nodes + [start_node]
        if is_valid_path(path, matrix, collecte):
            population.append(path)
    
    return population

# Calcul du coût d'un chemin donné
def calculate_cost(path, matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += matrix[path[i] - 1, path[i+1] - 1]  # ajusté pour indexation à partir de 1
    return cost

# Sélection des meilleurs chemins (tournoi)
def selection(population, matrix, tournament_size=5):
    population_with_cost = [(path, calculate_cost(path, matrix)) for path in population]
    population_with_cost.sort(key=lambda x: x[1])  # Trier par coût (croissant)
    # Tournoi : sélectionne un sous-ensemble de la population et choisit le meilleur
    selected = []
    for _ in range(len(population) // 2):
        tournament = random.sample(population_with_cost, tournament_size)
        best_in_tournament = min(tournament, key=lambda x: x[1])
        selected.append(best_in_tournament[0])
    return selected

# Croisement (crossover) de deux chemins
def crossover(parent1, parent2):
    size = len(parent1)
    cxpoint = random.randint(1, size - 1)
    child1 = parent1[:cxpoint] + [x for x in parent2 if x not in parent1[:cxpoint]]
    child2 = parent2[:cxpoint] + [x for x in parent1 if x not in parent2[:cxpoint]]
    return child1, child2

# Mutation d'un chemin (permutation de deux villes)
def mutate(path):
    size = len(path)
    i, j = random.sample(range(1, size - 1), 2)
    path[i], path[j] = path[j], path[i]
    return path

# Recherche Tabou : Amélioration locale des chemins
def tabu_search_local_optimization(population, matrix, tabu_list_size=10):
    new_population = []
    tabu_list = set()
    
    for path in population:
        best_path = path
        best_cost = calculate_cost(path, matrix)
        
        # Rechercher une solution voisine
        for i in range(len(path) - 1):
            for j in range(i + 1, len(path)):
                new_path = path[:]
                new_path[i], new_path[j] = new_path[j], new_path[i]
                
                if tuple(new_path) not in tabu_list and is_valid_path(new_path, matrix, collecte):
                    new_cost = calculate_cost(new_path, matrix)
                    if new_cost < best_cost:
                        best_path = new_path
                        best_cost = new_cost
        
        # Mettre à jour la liste Tabou
        tabu_list.add(tuple(best_path))
        if len(tabu_list) > tabu_list_size:
            tabu_list = set(list(tabu_list)[-tabu_list_size:])
        
        new_population.append(best_path)
    
    return new_population

# Fonction principale avec hybridation AG + Recherche Tabou
def solve_problem(matrix, collecte, population_size=10, generations=5):
    population = generate_initial_population(matrix, collecte, population_size)
    
    for generation in range(generations):
        print(f"Génération {generation + 1} - Meilleur coût : {calculate_cost(population[0], matrix)}")
        
        # Sélection
        selected_population = selection(population, matrix)
        
        # Croisement
        offspring = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])
        
        # Mutation
        mutated_offspring = [mutate(child) for child in offspring]
        
        # Amélioration avec Recherche Tabou
        improved_population = tabu_search_local_optimization(mutated_offspring, matrix)
        
        # Remplacer la population actuelle par la nouvelle population améliorée
        population = selected_population + improved_population
        population = sorted(population, key=lambda x: calculate_cost(x, matrix))[:population_size]
    
    best_path = population[0]
    best_cost = calculate_cost(best_path, matrix)
    
    return best_path, best_cost

# Exemple d'utilisation
file_path = 'data/matrice_routes_ordre.csv'  # Remplacez par le chemin de votre fichier CSV
matrix, collecte = load_adjacency_matrix_and_collecte(file_path)

best_path, best_cost = solve_problem(matrix, collecte)

print(f"Meilleur chemin trouvé : {best_path}")
print(f"Coût total : {best_cost}")


KeyboardInterrupt: 

In [25]:
import random
import numpy as np

# Charger la matrice d'adjacence et les points de collecte à partir du fichier CSV
def load_adjacency_matrix_and_collecte(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Supprimer les lignes vides
    lines = [line.strip() for line in lines if line.strip()]
    
    # La dernière ligne contient les points de collecte (exemple : 3;5)
    points_line = lines[-1]
    points_collecte = list(map(int, points_line.split(';')))
    
    # La matrice d'adjacence est dans toutes les autres lignes
    matrix = []
    for line in lines[:-1]:
        try:
            row = list(map(int, line.split(';')))
            matrix.append(row)
        except ValueError:
            print(f"Erreur de format dans la ligne : {line}. Elle sera ignorée.")
    
    return np.array(matrix), points_collecte

# Variables globales
start_node = 1  # On commence avec la ville 1

# Fonction de validation des chemins
def is_valid_path(path, matrix, collecte):
    if len(set(path)) != len(matrix):  # Il doit y avoir une ville pour chaque nœud
        return False
    if path[0] != start_node:  # doit commencer par la ville 1
        return False
    if path[-1] != start_node:  # doit finir par la ville 1
        return False
    if path[1] not in collecte:  # Le deuxième point doit être un point de collecte
        return False
    if path[-2] in collecte:  # L'avant-dernière ville ne doit pas être un point de collecte
        return False
    for i in range(len(path) - 1):
        if matrix[path[i] - 1][path[i+1] - 1] == -1:  # chemin bloqué (ajusté pour indexation à partir de 1)
            return False
    return True

# Génération d'une population initiale
def generate_initial_population(matrix, collecte, size=20):
    global start_node
    n = len(matrix)
    population = []
    
    while len(population) < size:
        # Créer une liste des villes disponibles (excluant la ville 0)
        remaining_nodes = list(range(1, n))  # Commence à 1 pour éviter la ville 0
        random.shuffle(remaining_nodes)
        path = [start_node] + remaining_nodes + [start_node]
        if is_valid_path(path, matrix, collecte):
            population.append(path)
    
    return population

# Calcul du coût d'un chemin donné
def calculate_cost(path, matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += matrix[path[i] - 1][path[i+1] - 1]  # ajusté pour indexation à partir de 1
    return cost

# Sélection des meilleurs chemins (tournoi)
def selection(population, matrix):
    population_with_cost = [(path, calculate_cost(path, matrix)) for path in population]
    population_with_cost.sort(key=lambda x: x[1])  # Trier par coût (croissant)
    return [path for path, cost in population_with_cost[:len(population) // 2]]

# Croisement (crossover) de deux chemins
def crossover(parent1, parent2):
    size = len(parent1)
    cxpoint = random.randint(1, size - 1)
    child1 = parent1[:cxpoint] + [x for x in parent2 if x not in parent1[:cxpoint]]
    child2 = parent2[:cxpoint] + [x for x in parent1 if x not in parent2[:cxpoint]]
    return child1, child2

# Mutation d'un chemin (permutation de deux villes)
def mutate(path):
    size = len(path)
    i, j = random.sample(range(1, size - 1), 2)
    path[i], path[j] = path[j], path[i]
    return path

# Fonction principale sans Recherche Tabou
def solve_problem(matrix, collecte, population_size=20, generations=10):
    population = generate_initial_population(matrix, collecte, population_size)
    
    for generation in range(generations):
        print(f"Génération {generation + 1} - Meilleur coût : {calculate_cost(population[0], matrix)}")
        
        # Sélection
        selected_population = selection(population, matrix)
        
        # Croisement
        offspring = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])
        
        # Mutation
        mutated_offspring = [mutate(child) for child in offspring]
        
        # Remplacer la population actuelle par la nouvelle population
        population = selected_population + mutated_offspring
        population = sorted(population, key=lambda x: calculate_cost(x, matrix))[:population_size]
    
    best_path = population[0]
    best_cost = calculate_cost(best_path, matrix)
    
    return best_path, best_cost

# Exemple d'utilisation
file_path = 'data/matrice_routes_ordre.csv'  # Remplacez par le chemin de votre fichier CSV
matrix, collecte = load_adjacency_matrix_and_collecte(file_path)

best_path, best_cost = solve_problem(matrix, collecte)

print(f"Meilleur chemin trouvé : {best_path}")
print(f"Coût total : {best_cost}")


KeyboardInterrupt: 

In [ ]:
import random
import numpy as np
from typing import List, Tuple, Set
import time

def load_adjacency_matrix_and_collecte(file_path: str) -> Tuple[np.ndarray, List[int]]:
    """Charge la matrice d'adjacence et les points de collecte depuis un fichier CSV.
    
    Args:
        file_path: Chemin vers le fichier CSV contenant la matrice et les points de collecte.
        
    Returns:
        Un tuple contenant (matrice d'adjacence, liste des points de collecte).
    """
    try:
        with open(file_path, 'r') as f:
            lines = [line.strip() for line in f if line.strip()]
            
        # Dernière ligne contient les points de collecte
        points_collecte = list(map(int, lines[-1].split(';')))
        
        # Matrice d'adjacence dans les autres lignes
        matrix = []
        for line in lines[:-1]:
            try:
                matrix.append(list(map(int, line.split(';'))))
            except ValueError as e:
                print(f"Erreur de conversion dans la ligne: {line}. Erreur: {e}")
                continue
                
        return np.array(matrix), points_collecte
        
    except FileNotFoundError:
        print(f"Erreur: Fichier {file_path} introuvable.")
        raise
    except Exception as e:
        print(f"Erreur inattendue lors du chargement du fichier: {e}")
        raise

class PathValidator:
    """Classe pour valider les chemins selon les contraintes du problème."""
    
    def __init__(self, matrix: np.ndarray, collecte_points: List[int], start_node: int = 1):
        self.matrix = matrix
        self.collecte = collecte_points
        self.start_node = start_node
        self.num_cities = len(matrix)
        
    def is_valid_path(self, path: List[int]) -> bool:
        """Vérifie si un chemin satisfait toutes les contraintes."""
        return (self._has_all_cities(path) and 
                self._starts_and_ends_correctly(path) and 
                self._collecte_constraints(path) and 
                self._no_blocked_paths(path))
    
    def _has_all_cities(self, path: List[int]) -> bool:
        """Vérifie que le chemin visite toutes les villes exactement une fois."""
        return len(set(path)) == self.num_cities
    
    def _starts_and_ends_correctly(self, path: List[int]) -> bool:
        """Vérifie que le chemin commence et se termine au bon nœud."""
        return path[0] == self.start_node and path[-1] == self.start_node
    
    def _collecte_constraints(self, path: List[int]) -> bool:
        """Vérifie les contraintes spécifiques aux points de collecte."""
        return (path[1] in self.collecte and 
                path[-2] not in self.collecte)
    
    def _no_blocked_paths(self, path: List[int]) -> bool:
        """Vérifie qu'il n'y a pas de chemins bloqués (-1) entre les villes."""
        for i in range(len(path) - 1):
            if self.matrix[path[i] - 1, path[i+1] - 1] == -1:
                return False
        return True

def generate_initial_population(matrix: np.ndarray, collecte: List[int], 
                              population_size: int = 20, 
                              start_node: int = 1) -> List[List[int]]:
    """Génère une population initiale de chemins valides."""
    validator = PathValidator(matrix, collecte, start_node)
    n = len(matrix)
    population = []
    attempts = 0
    max_attempts = population_size * 100  # Limite pour éviter une boucle infinie
    
    while len(population) < population_size and attempts < max_attempts:
        # Génère un chemin aléatoire qui commence et finit par start_node
        remaining_nodes = [x for x in range(1, n+1) if x != start_node]
        random.shuffle(remaining_nodes)
        path = [start_node] + remaining_nodes + [start_node]
        
        if validator.is_valid_path(path):
            population.append(path)
        attempts += 1
        
    if len(population) < population_size:
        print(f"Avertissement: Seulement {len(population)} chemins valides générés sur {population_size} demandés")
        
    return population

def calculate_cost(path: List[int], matrix: np.ndarray) -> int:
    """Calcule le coût total d'un chemin."""
    return sum(matrix[path[i]-1, path[i+1]-1] for i in range(len(path)-1))

def tournament_selection(population: List[List[int]], matrix: np.ndarray, 
                        tournament_size: int = 5, 
                        selection_size: int = None) -> List[List[int]]:
    """Sélectionne les meilleurs chemins par tournoi."""
    if not population:
        return []
        
    if not selection_size:
        selection_size = len(population) // 2
        
    # S'assurer que tournament_size n'est pas plus grand que la population
    tournament_size = min(tournament_size, len(population))
    
    population_with_cost = [(path, calculate_cost(path, matrix)) for path in population]
    selected = []
    
    for _ in range(selection_size):
        # Sélection aléatoire des participants au tournoi
        tournament = random.sample(population_with_cost, tournament_size)
        # Sélection du meilleur (coût le plus bas)
        best_path = min(tournament, key=lambda x: x[1])[0]
        selected.append(best_path)
        
    return selected


def ordered_crossover(parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
    """Croisement ordonné pour préserver les permutations valides."""
    size = len(parent1)
    start, end = sorted(random.sample(range(1, size-1), 2))
    
    def create_child(p1, p2):
        child = [None]*size
        child[0] = child[-1] = p1[0]  # start and end with start_node
        
        # Copier la section entre start et end de p1
        child[start:end] = p1[start:end]
        
        # Remplir le reste avec les éléments de p2 dans l'ordre, en évitant les doublons
        remaining = [item for item in p2 if item not in child[start:end]]
        ptr = 1
        for i in range(1, size-1):
            if child[i] is None:
                child[i] = remaining[ptr-1]
                ptr += 1
                
        return child
    
    child1 = create_child(parent1, parent2)
    child2 = create_child(parent2, parent1)
    
    return child1, child2

def swap_mutation(path: List[int], mutation_rate: float = 0.1) -> List[int]:
    """Mutation par échange de deux villes (sauf la première et dernière)."""
    if random.random() < mutation_rate and len(path) > 3:
        i, j = random.sample(range(1, len(path)-1), 2)
        path[i], path[j] = path[j], path[i]
    return path

class TabuSearch:
    """Implémentation de la recherche Tabou pour l'optimisation locale."""
    
    def __init__(self, matrix: np.ndarray, collecte: List[int], max_iter: int = 100, 
                 tabu_size: int = 10, start_node: int = 1):
        self.matrix = matrix
        self.validator = PathValidator(matrix, collecte, start_node)
        self.max_iter = max_iter
        self.tabu_size = tabu_size
        self.start_node = start_node
        
    def optimize(self, path: List[int]) -> List[int]:
        """Optimise un chemin en utilisant la recherche Tabou."""
        best_path = path.copy()
        best_cost = calculate_cost(best_path, self.matrix)
        current_path = path.copy()
        tabu_list = set()
        
        for _ in range(self.max_iter):
            neighbors = self._generate_neighbors(current_path)
            next_path = None
            next_cost = float('inf')
            
            for neighbor in neighbors:
                neighbor_tuple = tuple(neighbor)
                if neighbor_tuple not in tabu_list and self.validator.is_valid_path(neighbor):
                    cost = calculate_cost(neighbor, self.matrix)
                    if cost < next_cost:
                        next_path = neighbor
                        next_cost = cost
            
            if not next_path:  # Tous les voisins sont dans la liste Tabou
                break
                
            current_path = next_path
            tabu_list.add(tuple(next_path))
            
            # Garder une trace du meilleur chemin trouvé
            if next_cost < best_cost:
                best_path = next_path
                best_cost = next_cost
                
            # Limiter la taille de la liste Tabou
            if len(tabu_list) > self.tabu_size:
                tabu_list.pop()
                
        return best_path
    
    def _generate_neighbors(self, path: List[int]) -> List[List[int]]:
        """Génère des voisins en échangeant deux villes (sauf la première et dernière)."""
        neighbors = []
        for i in range(1, len(path)-2):
            for j in range(i+1, len(path)-1):
                neighbor = path.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        return neighbors

def solve_problem(matrix: np.ndarray, collecte: List[int], 
                  population_size: int = 20, generations: int = 50,
                  mutation_rate: float = 0.1, elite_size: int = 2,
                  start_node: int = 1) -> Tuple[List[int], int]:
    """Résout le problème d'optimisation avec un algorithme génétique hybride."""
    validator = PathValidator(matrix, collecte, start_node)
    tabu_search = TabuSearch(matrix, collecte)
    
    # Génération de la population initiale
    population = generate_initial_population(matrix, collecte, population_size, start_node)
    if not population:
        raise ValueError("Impossible de générer une population initiale valide. Vérifiez les contraintes.")
    
    population.sort(key=lambda x: calculate_cost(x, matrix))
    
    best_global_path = population[0]
    best_global_cost = calculate_cost(best_global_path, matrix)
    
    for generation in range(generations):
        start_time = time.time()
        
        # Sélection élitiste - garder les meilleurs solutions
        elites = population[:elite_size]
        
        # Sélection par tournoi
        selected = tournament_selection(population, matrix)
        if not selected:
            print("Avertissement: Aucun individu sélectionné. Utilisation de la population actuelle.")
            selected = population.copy()
        
        # Croisement
        offspring = []
        for i in range(0, len(selected)-1, 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = ordered_crossover(parent1, parent2)
            offspring.extend([child1, child2])
        
        # Mutation
        mutated_offspring = [swap_mutation(child, mutation_rate) for child in offspring]
        
        # Évaluation et filtrage des solutions invalides
        valid_offspring = [path for path in mutated_offspring if validator.is_valid_path(path)]
        
        # Optimisation locale avec recherche Tabou (sur une partie de la population)
        to_optimize = valid_offspring[:max(1, len(valid_offspring)//2)]  # Au moins 1 élément
        optimized = [tabu_search.optimize(path) for path in to_optimize]
        
        # Création de la nouvelle population
        new_population = elites + optimized + valid_offspring[len(valid_offspring)//2:]
        new_population.sort(key=lambda x: calculate_cost(x, matrix))
        population = new_population[:population_size]
        
        # Mise à jour de la meilleure solution globale
        if population:  # Vérifier que la population n'est pas vide
            current_best_cost = calculate_cost(population[0], matrix)
            if current_best_cost < best_global_cost:
                best_global_path = population[0]
                best_global_cost = current_best_cost
        
        # Affichage des statistiques
        gen_time = time.time() - start_time
        avg_cost = sum(calculate_cost(p, matrix) for p in population) / len(population) if population else 0
        print(f"Génération {generation+1}: Meilleur={best_global_cost}, Moyen={avg_cost:.1f}, Temps={gen_time:.2f}s")
    
    return best_global_path, best_global_cost

if __name__ == "__main__":
    try:
        file_path = 'data/matrice_routes_ordre.csv'
        print(f"Chargement de la matrice depuis {file_path}...")
        matrix, collecte = load_adjacency_matrix_and_collecte(file_path)
        
        print("\nParamètres du problème:")
        print(f"- Nombre de villes: {len(matrix)}")
        print(f"- Points de collecte: {collecte}")
        print(f"- Taille de la matrice: {matrix.shape}")
        
        print("\nDébut de l'optimisation...")
        start_time = time.time()
        best_path, best_cost = solve_problem(
            matrix, collecte,
            population_size=30,
            generations=100,
            mutation_rate=0.15
        )
        
        print(f"\nOptimisation terminée en {time.time()-start_time:.2f} secondes")
        print(f"\nMeilleur chemin trouvé: {best_path}")
        print(f"Coût total: {best_cost}")
        
    except Exception as e:
        print(f"Une erreur s'est produite: {e}")

Chargement de la matrice depuis data/matrice_routes_ordre.csv...

Paramètres du problème:
- Nombre de villes: 10
- Points de collecte: [3, 5]
- Taille de la matrice: (10, 10)

Début de l'optimisation...
Génération 1: Meilleur=833, Moyen=846.5, Temps=0.00s
Une erreur s'est produite: Sample larger than population or is negative
